# Deep Learning & Applied AI

# Tutorial 2: Tensors operations

In this tutorial, we will cover:

- Tensors operations: broadcasting, (not)-elementwise operations, tensors contraction, einsum

Our info:

- Luca Moschella (moschella@di.uniroma1.it)
- Antonio Norelli (norelli@di.uniroma1.it)

Course:

- Website and notebooks will be available at [DLAI-s2-2021](https://erodola.github.io/DLAI-s2-2021/)



## Introduction

In this tutorial we will continue to learn basic tensor usage, we will cover  the broadcasting behaviour, fundamental linear algebra operations, and finally `einsum`, a single operation implementing the Einstein notation to rule them all!

All these tensor operations will come in handy to build our deep neural networks.
Yet, the high level API offered by PyTorch to perform GPU-acccelerated linear algebra operations may turn useful in many other fields, from microbiology to fluid dynamics. 

The GPU computing paradigm is recent and offers several benefits respect to single core machines or traditional supercomputers equipped with many single-core nodes.
Deep learning frameworks such as the one we are studying are a very good compromise between simplicity and expressivenes to unleash the power of GPU-computing.

To get even more control you should tackle directly CUDA.

## PyTorch

You should familiarize with the [PyTorch Documentation](https://pytorch.org/docs/stable/) as it will greatly assist you.






In [1]:
import torch
import numpy as np

In [2]:
from typing import Union

# Utility print function
def print_arr(*arr: Union[torch.Tensor, np.ndarray], prefix: str = "") -> None:
    """Pretty print tensors, together with their shape and type

    :param arr: one or more tensors
    :param prefix: prefix to use when printing the tensors
    """
    print(
        "\n\n".join(
            f"{prefix}{str(x)} <shape: {x.shape}> <dtype: {x.dtype}>" for x in arr
        )
    )

####Set torch and numpy random seeds for reproducibility
If you are going to use a gpu, two further options must be set. (CuDNN is a library of CUDA for Deep Neural Networks).


> Once your model works, remember to test without a fixed seed! You may have overfitted the seed (e.g. you have chosen hyperparameters that happens to work particularly well with a given seed.)

In [3]:
import random

np.random.seed(42)
random.seed(0)

torch.manual_seed(0)
torch.cuda.manual_seed_all(0)
torch.backends.cudnn.deterministic = True  # Note that this Deterministic mode can have a performance impact
torch.backends.cudnn.benchmark = False

# We will see frameworks that aid the reproducibility of your code, 
# e.g. PyTorch Lightning exposes a `seed_everything` function by default:
# https://github.com/PyTorchLightning/pytorch-lightning/blob/e1f5eacab98670bc1de72c88657404a15aadd527/pytorch_lightning/utilities/seed.py#L29

### **Tensor operations**



In [4]:
t = torch.rand(3,3)
t

tensor([[0.4963, 0.7682, 0.0885],
        [0.1320, 0.3074, 0.6341],
        [0.4901, 0.8964, 0.4556]])

Functions that operate on tensors are often accessible in different ways: 

- From the **`torch` module**...:

In [5]:
torch.add(t, t)

tensor([[0.9925, 1.5364, 0.1770],
        [0.2641, 0.6148, 1.2682],
        [0.9802, 1.7929, 0.9113]])

- ...or by tensors **methods**:

In [6]:
t.add(t)

tensor([[0.9925, 1.5364, 0.1770],
        [0.2641, 0.6148, 1.2682],
        [0.9802, 1.7929, 0.9113]])

- ...or through **overloaded** operators:




In [7]:
t + t

tensor([[0.9925, 1.5364, 0.1770],
        [0.2641, 0.6148, 1.2682],
        [0.9802, 1.7929, 0.9113]])

In [8]:
# t is unchanged
t

tensor([[0.4963, 0.7682, 0.0885],
        [0.1320, 0.3074, 0.6341],
        [0.4901, 0.8964, 0.4556]])

These functions are all equivalent, they are *aliases* of the same method.
Personal preference, code consistency, and readability should guide your decision of which one to use. 

> e.g. `torch.add(...)` may be too verbose, but in some cases it may be preferable since it makes explicit to the code-reader that you are dealing with tensors. Eventhough, if you are using [types](https://docs.python.org/3/library/typing.html) -- and you should be using types -- it will be rarely necessary.


Most operation in PyTorch are **not in-place**. It means that the resulting tensor is a *new* tensor, that does not share the underlying data with other tensors. Changes to the new tensor are not reflected to other tensors. 
We will see in future tutorials why this is important, for now a *tldr* is: in-place operations may break the auto-differentiation mechanism.


In-place operations are available in PyTorch, and in some cases can be useful: they are more efficient since they never require to perform copies of the data (e.g. if you do not need the autodifferentiation).
They are normally recognizable by a trailing `_`:

In [9]:
t

tensor([[0.4963, 0.7682, 0.0885],
        [0.1320, 0.3074, 0.6341],
        [0.4901, 0.8964, 0.4556]])

In [10]:
t.add_(t)

tensor([[0.9925, 1.5364, 0.1770],
        [0.2641, 0.6148, 1.2682],
        [0.9802, 1.7929, 0.9113]])

In [11]:
t  # t itself changed!

tensor([[0.9925, 1.5364, 0.1770],
        [0.2641, 0.6148, 1.2682],
        [0.9802, 1.7929, 0.9113]])

Another common in-place operation is the assignment:

In [12]:
t[0] = 42
t

tensor([[42.0000, 42.0000, 42.0000],
        [ 0.2641,  0.6148,  1.2682],
        [ 0.9802,  1.7929,  0.9113]])

#### **Basic operations and broadcasting**

Basic mathematical operations $(+, -, *, /, **)$ are applied **elementwise or** do **broadcasting**:

In [13]:
x = torch.tensor([[1, 2], [3, 4]], dtype=torch.float64)
y = torch.tensor([[5, 6], [7, 8]], dtype=torch.float64)

print(x + y)  # elementwise sum
print(x + 4.2)  # broadcasting

tensor([[ 6.,  8.],
        [10., 12.]], dtype=torch.float64)
tensor([[5.2000, 6.2000],
        [7.2000, 8.2000]], dtype=torch.float64)


In [14]:
# other examples
print(x * y - 5)
print((x - y) / y)

tensor([[ 0.,  7.],
        [16., 27.]], dtype=torch.float64)
tensor([[-0.8000, -0.6667],
        [-0.5714, -0.5000]], dtype=torch.float64)


Broadcasting is even more powerful...

In [15]:
m = torch.arange(12).reshape(4, 3)
v = torch.tensor([100, 0, 100])
n = m + v
print_arr(m, v, n)

tensor([[ 0,  1,  2],
        [ 3,  4,  5],
        [ 6,  7,  8],
        [ 9, 10, 11]]) <shape: torch.Size([4, 3])> <dtype: torch.int64>

tensor([100,   0, 100]) <shape: torch.Size([3])> <dtype: torch.int64>

tensor([[100,   1, 102],
        [103,   4, 105],
        [106,   7, 108],
        [109,  10, 111]]) <shape: torch.Size([4, 3])> <dtype: torch.int64>


In [16]:
m = torch.arange(12).reshape(4, 3)
u = torch.tensor([0, 10, 0, 10]).reshape(4,1)
n = m + u
print_arr(m, u, n)

tensor([[ 0,  1,  2],
        [ 3,  4,  5],
        [ 6,  7,  8],
        [ 9, 10, 11]]) <shape: torch.Size([4, 3])> <dtype: torch.int64>

tensor([[ 0],
        [10],
        [ 0],
        [10]]) <shape: torch.Size([4, 1])> <dtype: torch.int64>

tensor([[ 0,  1,  2],
        [13, 14, 15],
        [ 6,  7,  8],
        [19, 20, 21]]) <shape: torch.Size([4, 3])> <dtype: torch.int64>


In [17]:
w = u + v
print_arr(u, v, w)

tensor([[ 0],
        [10],
        [ 0],
        [10]]) <shape: torch.Size([4, 1])> <dtype: torch.int64>

tensor([100,   0, 100]) <shape: torch.Size([3])> <dtype: torch.int64>

tensor([[100,   0, 100],
        [110,  10, 110],
        [100,   0, 100],
        [110,  10, 110]]) <shape: torch.Size([4, 3])> <dtype: torch.int64>


Mastering broadcasting is very useful to write **vectorized** code, i.e. code that avoids explicit python loops which are slow and can not be efficiently parallelized. 

Instead, this approach takes advantage of the underlying C implementation of PyTorch and Numpy (on CPU) or CUDA implementation of Pytorch (on GPU).

![broadcasting](https://jakevdp.github.io/PythonDataScienceHandbook/figures/02.05-broadcasting.png)

##### **EXERCISE**
>
> Given two vectors $X \in R^n$ and $Y \in R^m$ compute the differences between all possible pairs of numbers, and organize those differences in a matrix $Z \in R^{n \times m}$:
> $$ z_{ij} = x_i - y_j $$


In [18]:
# ✏️ your code here 

##### **EXERCISE**
>
> Given a  ${n \times m}$ tensor and two indices $a \in [0, n)$, $b \in [0, m)$ and $p \in [0, +\inf)$,
> create a new tensor $Y \in R^{n \times m}$ such that:
>
> $$ y_{ij} = d_{L_p}( (i,j), (a,b) ) \text{ for each }  i \in [0, n), j \in  [0, m) $$
>
> That is, consider pairs of indices as points in $R^2$. Try different values of $p$ to see what happens.
>
> e.g. Using the $L_1$ distance given $(i,j) = (3, 5)$ and $(a,b) = (14, 20)$ we get:
> $$ y_{3,5} = d_{L_1}( (3, 5), (14, 20) ) = |3 - 14| + |5 - 20| $$
>
> With $L_p$ we refer to the [*p*-norm](https://en.wikipedia.org/wiki/Lp_space#The_p-norm_in_finite_dimensions)

In [19]:
# Utility function
import plotly.express as px

def plot_row_images(images: Union[torch.Tensor, np.ndarray]) -> None:
  """ Plots the images in a subplot with multiple rows.

  Handles correctly grayscale images.

  :param images: tensor with shape [number of images, width, height, <colors>]
  """
  from plotly.subplots import make_subplots
  import plotly.graph_objects as go
  fig = make_subplots(rows=1, cols=images.shape[0] ,
                      specs=[[{}] * images.shape[0]])
  
  # Convert grayscale image to something that go.Image likes
  if images.dim() == 3:
    images = torch.stack((images, images, images), dim= -1)
  elif (images.dim() == 4 and images.shape[-1] == 1):
    images = torch.cat((images, images, images), dim= -1)

  assert images.shape[-1] == 3 or images.shape[-1] == 4
    
  for i in range(images.shape[0]):  
    i_image = np.asarray(images[i, ...])

    fig.add_trace( 
        go.Image(z = i_image, zmin=[0, 0, 0, 0], zmax=[1, 1, 1, 1]),
        row=1, col=i + 1
    )

  fig.show()


# When using plotly pay attention that often it does not like PyTorch Tensors
# ...and it does not give any error, just a empty plot.

In [20]:
x = torch.zeros(300, 300)
a = 150
b = 150

x[a, b] = 1  # Just to visualize the starting point
plot_row_images(x[None, :])

In [21]:
# ✏️ your code here 

> Follow up: does it work with `p = 10`?

##### **Broadcasting, let's take a peek under the hood**

In short: if a PyTorch operation supports broadcast, then **its Tensor arguments can be automatically expanded to be of equal sizes** (without making copies of the data).

###### **Broadcastable tensors**

Two tensors are "broadcastable" if:
- Each tensor has at least one dimension
- When iterating over the dimension sizes, starting at the trailing dimension, the dimension **sizes** must either **be equal**, **one of them is 1**, or **one of them does not exist**.


###### **Broadcasting rules**

Broadcasting two tensors together follows these rules:

1. All input tensors have **1's prepended to their shapes**, to match the rank of the biggest tensor in input
2. The size in each dimension of the **output shape** is the maximum of all the input sizes in that dimension
3. An input can be used in the computation if its size in a particular **dimension either match** the output size in that dimension, **or has value exactly 1**
4. If an input has a dimension size of 1 in its shape, the **first data entry in that dimension will be used for all calculations** along that dimension. 

**In our example**:

- `m` has shape `[4, 3]`
- `v` has shape `[3,]`.


In [22]:
print_arr(m, v)

tensor([[ 0,  1,  2],
        [ 3,  4,  5],
        [ 6,  7,  8],
        [ 9, 10, 11]]) <shape: torch.Size([4, 3])> <dtype: torch.int64>

tensor([100,   0, 100]) <shape: torch.Size([3])> <dtype: torch.int64>



Following the Broadcasting logic, this is what happened:

- `v` has less dims than `m` so a dimension of `1` is **prepended** $\to$ `v` is now `[1, 3]`.
- Output shape will be `[max(1, 4), max(3, 3)] = [4, 3]`.
- `dim 1` of `v` matches exactly `3`; `dim 0` is `1`, so we can use the first data entry in that dimension (i.e. the whole `row 0` of `v`) each time any row is accessed. This is effectively like converting `v` from `[1, 3]` to `[4, 3]` by replicating.


For more on broadcasting, see the [documentation](https://docs.scipy.org/doc/numpy/user/basics.broadcasting.html).

Functions that support broadcasting are known as universal functions (i.e. ufuncs). For Numpy you can find the list of all universal functions in the [documentation](https://docs.scipy.org/doc/numpy/reference/ufuncs.html#available-ufuncs).

#### **Non-elementwise operations**


PyTorch and NumPy provide many useful functions to perform computations on tensors:

In [23]:
x = torch.tensor([[1, 2], [3, 4]], dtype=torch.float32)
print_arr(x)

tensor([[1., 2.],
        [3., 4.]]) <shape: torch.Size([2, 2])> <dtype: torch.float32>


In [24]:
# Sum up all the elements
print_arr(torch.sum(x))

tensor(10.) <shape: torch.Size([])> <dtype: torch.float32>


In [25]:
# Compute the mean of each column
print_arr(torch.mean(x, dim=0))

tensor([2., 3.]) <shape: torch.Size([2])> <dtype: torch.float32>


> **REMEMBER!**
>
> In order to avoid confusion with the `dim` parameter, you can think of it as an **index over the list returned by `tensor.shape`**. The operation is performed iterating over that dimension.
> 
> Visually: 
> 
><img src="https://qph.fs.quoracdn.net/main-qimg-30be20ab9458b5865b526d287b4fef9a" width="500" >

In [26]:
# Compute the product of each row
print_arr(torch.prod(x, dim=1))

tensor([ 2., 12.]) <shape: torch.Size([2])> <dtype: torch.float32>


In [27]:
# Max along the rows (i.e. max value in each column)
values, indices = torch.max(x, dim=0)
print_arr(values)

tensor([3., 4.]) <shape: torch.Size([2])> <dtype: torch.float32>


In [28]:
# Max along the columns (i.e. max value in each row)
values, indices = torch.max(x, dim=1)
print_arr(values)

tensor([2., 4.]) <shape: torch.Size([2])> <dtype: torch.float32>


###### **Dim parameter, let's take a peek under the hood**


Let's see what exactly does the `dim` parameter:

In [29]:
dim = 2

a = torch.rand(2, 3, 4)
out = a.sum(dim=dim)
out

tensor([[1.4052, 1.6789, 1.9249],
        [2.6059, 1.7270, 1.7864]])

In [30]:
# It is summing over the `dim` dimension, i.e.:
a.shape

torch.Size([2, 3, 4])

In [31]:
# The `dim` dimension has 4 elements
a.shape[dim]

4

In [32]:
# The dimension dim collapses, the output tensor will have shape:
new_shape = a.shape[:dim] + a.shape[dim + 1:]
new_shape

torch.Size([2, 3])

In [33]:
# Explicitly compute the sum over dim
out = torch.zeros(new_shape)                  
for i in range(a.shape[dim]):
  out += a.select(dim=dim, index=i)

  # In this case, since `dim` is the last dimension of a
  # you could use the notation:
  # out += a[..., i]
  # With this notation, while beeing extremely useful to handle
  # tensors with a variable number of dimensions, you cannot 
  # choose an intermediate dim like with the `select`.

out

# **DO NOT** use for loops in your code

tensor([[1.4052, 1.6789, 1.9249],
        [2.6059, 1.7270, 1.7864]])

##### **EXERCISE**
>
> Given a matrix $X \in R^{k \times k}$ compute the mean of the values along its diagonal. Perform this computation in at least two different ways, then check that the result is the same.

In [34]:
x = torch.rand(4, 4)
print_arr(x)

tensor([[0.2698, 0.1507, 0.0317, 0.2081],
        [0.9298, 0.7231, 0.7423, 0.5263],
        [0.2437, 0.5846, 0.0332, 0.1387],
        [0.2422, 0.8155, 0.7932, 0.2783]]) <shape: torch.Size([4, 4])> <dtype: torch.float32>


In [35]:
# ✏️ your code here 

##### **EXERCISE**
>
> Given a binary non-symmetric matrix $X \in \{0, 1\}^{n, n}$, build the symmetric matrix $Y \in \{0, 1\}^{n, n}$ defined as:
> $$
y_{ij} =
\begin{cases}
1 & \text{if } x_{ij} = 1 \\
1 & \text{if } x_{ji} = 1 \\
0 & \text{otherwise}
\end{cases}
$$ 
>
> *Hint*: search for `clamp` in the [docs](https://pytorch.org/docs/stable/index.html)

In [36]:
x = torch.randint(0, 2, (5, 5))  # Non-symmetric matrix
x

tensor([[1, 0, 1, 1, 1],
        [1, 0, 1, 0, 0],
        [1, 1, 0, 1, 0],
        [1, 0, 0, 0, 0],
        [0, 1, 1, 0, 0]])

In [37]:
# ✏️ your code here

#### **Tensor contractions**

##### **Matrix multiplication**

Given $X \in R^{n \times d}$ and $Y \in R^{d \times v}$, their matrix multiplication $Z \in R^{n \times v}$ is defined as:

$$ \sum_{k=0}^{d} x_{ik} y_{kj} = z_{ij} $$


In [38]:
x = torch.tensor([[1, 2], [3, 4], [5, 6]])
y = torch.tensor([[1, 2], [2, 1]])
print_arr(x, y)

tensor([[1, 2],
        [3, 4],
        [5, 6]]) <shape: torch.Size([3, 2])> <dtype: torch.int64>

tensor([[1, 2],
        [2, 1]]) <shape: torch.Size([2, 2])> <dtype: torch.int64>


In [39]:
x @ y  # Operator overload

tensor([[ 5,  4],
        [11, 10],
        [17, 16]])

In [40]:
torch.mm(x, y)  # PyTorch function

tensor([[ 5,  4],
        [11, 10],
        [17, 16]])

In [41]:
x.mm(y)  # Tensor method

tensor([[ 5,  4],
        [11, 10],
        [17, 16]])

In [42]:
torch.einsum('ik, kj -> ij', (x, y))  # Einsum notation!

# It summed up dimension labeled with the index `k`

tensor([[ 5,  4],
        [11, 10],
        [17, 16]])

##### **Dot product** 
Also known as Inner product. 
Given $x \in R^k$ and $y \in R^k$, the dot product $z \in R$ is defined as:

$$ \sum_{i=0}^{k} x_i y_i = z $$

Unlike MATLAB, ``*`` is the element wise multiplication, not the matrix multiplication

In [43]:
x = torch.tensor([1, 2, 3])
y = torch.tensor([4, 5, 6])
print_arr(x, y)

tensor([1, 2, 3]) <shape: torch.Size([3])> <dtype: torch.int64>

tensor([4, 5, 6]) <shape: torch.Size([3])> <dtype: torch.int64>


In [44]:
# We want to perform:
(1 * 4) + (2 * 5) + (3 * 6)

32

In [45]:
torch.dot(x, y)  # PyTorch function

tensor(32)

In [46]:
x.dot(y) # Tensor method

tensor(32)

In [47]:
x @ y  # PyTorch operator overload

tensor(32)

In [48]:
torch.einsum('i, i ->', (x, y))  # Einstein notation!

# Multiply point-wise repeating indices in the input
# Sum up along the indices that `do not` appear in the output

tensor(32)

##### **Batch matrix multiplication**

Often we want to perform more operations together. Why?
- Reduce the **overhead of uploading** each tensor to/from the GPU memory
- **Better parallelization** of the computation

Given two 3D tensors, each one containing ``b`` matrices,
$X \in R^{b \times n \times m}$
and  
$Y \in R^{b \times m \times p}$, 

We want to multiply together each $i$-th couple of matrices, obtaining a tensor $Z \in R^{b \times n \times p}$ defined as:

$$ z_{bij} = \sum_{k=0}^m x_{bik} y_{bkj} $$

In [49]:
x = torch.tensor([[[1, 2], [3, 4], [5, 6]], [[1, 2], [3, 4], [5, 6]]])
y = torch.tensor([[[1, 2], [2, 1]], [[1, 2], [2, 1]]])
print_arr(x, y)

tensor([[[1, 2],
         [3, 4],
         [5, 6]],

        [[1, 2],
         [3, 4],
         [5, 6]]]) <shape: torch.Size([2, 3, 2])> <dtype: torch.int64>

tensor([[[1, 2],
         [2, 1]],

        [[1, 2],
         [2, 1]]]) <shape: torch.Size([2, 2, 2])> <dtype: torch.int64>


In [50]:
torch.bmm(x, y)  # **not** torch.mm

tensor([[[ 5,  4],
         [11, 10],
         [17, 16]],

        [[ 5,  4],
         [11, 10],
         [17, 16]]])

In [51]:
x @ y

tensor([[[ 5,  4],
         [11, 10],
         [17, 16]],

        [[ 5,  4],
         [11, 10],
         [17, 16]]])

In [52]:
torch.einsum('bik, bkj -> bij', (x, y)) # Einstein notation!

tensor([[[ 5,  4],
         [11, 10],
         [17, 16]],

        [[ 5,  4],
         [11, 10],
         [17, 16]]])

Can you feel the power of Einstein notation vibrating off of you?


![Surfing einstein](https://roma.corriere.it/methode_image/2019/05/13/Roma/Foto%20Roma%20-%20Trattate/einstein2-kZfE-U3120295526975hVE-656x492@Corriere-Web-Roma.JPG)

##### **Broadcast matrix multiplication**

Given $b$ matrices with dimensions $n \times m$ organized in one 3D tensor $X \in R^{b \times n \times m}$
and one 2D tensor $Y \in R^{m \times p}$, 

We want to multiply together each matrix $X_{i,:,:}$ with $Y$, obtaining a tensor $Z \in R^{b \times n \times p}$ defined as:

$$ z_{bij} = \sum_{k=0}^m x_{bik} y_{kj} $$


In [53]:
x = torch.tensor([[[1, 2], [3, 4], [5, 6]], [[1, 2], [3, 4], [5, 6]]])
y = torch.tensor([[1, 2], [2, 1]])
print_arr(x, y)

tensor([[[1, 2],
         [3, 4],
         [5, 6]],

        [[1, 2],
         [3, 4],
         [5, 6]]]) <shape: torch.Size([2, 3, 2])> <dtype: torch.int64>

tensor([[1, 2],
        [2, 1]]) <shape: torch.Size([2, 2])> <dtype: torch.int64>


In [54]:
x @ y   # Operator overload: always use the last two dimensions

tensor([[[ 5,  4],
         [11, 10],
         [17, 16]],

        [[ 5,  4],
         [11, 10],
         [17, 16]]])

In [55]:
torch.matmul(x, y)  # Explicit PyTorch function: always use the last two dimensions

tensor([[[ 5,  4],
         [11, 10],
         [17, 16]],

        [[ 5,  4],
         [11, 10],
         [17, 16]]])

##### **EXERCISE**
>
> Use the einsum notation to compute the equivalent broadcast matrix multiplication!

##### **Einsum notation**

Einstein notation is a way to express complex operations on tensors

- It is **concise but enough expressive** to do almost every operation you will need in building your neural networks, letting you think on the only thing that matters... **dimensions!**
- You will **not need to check your dimensions** after an einsum operation, since the dimensions themselves are *defining* the tensor operation.
- You will **not need shape-comment** your tensors. Those comments do not work: they are bound to get outdated.
-  You will not need to explicitly code **intermediate operations** such as reshaping, transposing and intermediate tensors
- It is **not library-specific**, being avaiable in ``numpy``, ``pytorch`` and ``tensorflow`` with the same signature. So you do not need to remember the functions signature in all the frameworks.
- Can sometimes be compiled to high-performing code (e.g. [Tensor Comprehensions](https://pytorch.org/blog/tensor-comprehensions/))

Check [this blog post by Olexa Bilaniuk](https://obilaniu6266h16.wordpress.com/2016/02/04/einstein-summation-in-numpy/) to take a peek under the hood of einsum and [this one by Tim Rocktäschel](https://rockt.github.io/2018/04/30/einsum) to look at even more examples than the ones that follows.

Its formal behaviour is well described in the [Numpy documentation](https://docs.scipy.org/doc/numpy/reference/generated/numpy.einsum.html).
However, it is very intuitive and better explained through examples.

![alt text](https://obilaniu6266h16.files.wordpress.com/2016/02/einsum-fmtstring.png?w=676)

> *Historical note (taken from the Bilaniuk post)*
>
> Einstein had no part in the development of this notation. He merely popularized it, by expressing his entire theory of General Relativity in it. In a letter to [Tullio Levi-Civita](https://en.wikipedia.org/wiki/Tullio_Levi-Civita), co-developer alongside [Gregorio Ricci-Curbastro](https://en.wikipedia.org/wiki/Gregorio_Ricci-Curbastro) of Ricci calculus (of which this summation notation was only a part), Einstein wrote:
>
> " *I admire the elegance of your method of computation; it must be nice to ride through these fields upon the horse of true mathematics while the like of us have to make our way laboriously on foot.* "

In [56]:
a = torch.arange(6).reshape(2, 3)

###### **Matrix transpose**

$$ B_{ji} = A_{ij} $$

In [57]:
# The characters are indices along each dimension
b = torch.einsum('ij -> ji', a)
print_arr(a, b)

tensor([[0, 1, 2],
        [3, 4, 5]]) <shape: torch.Size([2, 3])> <dtype: torch.int64>

tensor([[0, 3],
        [1, 4],
        [2, 5]]) <shape: torch.Size([3, 2])> <dtype: torch.int64>


###### **Sum**

$$ b = \sum_i \sum_j A_{ij} := A_{ij} $$


In [58]:
# Indices that do not appear in the output tensor are summed up
b = torch.einsum('ij -> ', a)
print_arr(a, b)

tensor([[0, 1, 2],
        [3, 4, 5]]) <shape: torch.Size([2, 3])> <dtype: torch.int64>

tensor(15) <shape: torch.Size([])> <dtype: torch.int64>


###### **Column sum**

$$ b_j = \sum_i A_{ij} := A_{ij} $$

In [59]:
# Indices that do not appear in the output tensor are summed up,
# even if some other index appears
b = torch.einsum('ij -> j', a)
print_arr(a, b)

tensor([[0, 1, 2],
        [3, 4, 5]]) <shape: torch.Size([2, 3])> <dtype: torch.int64>

tensor([3, 5, 7]) <shape: torch.Size([3])> <dtype: torch.int64>


###### **EXERCISE**
>
> Which will be the shape and type of the following tensor $X \in R^{100 \times 200}$? Which values will it contain? Why?

In [60]:
x = (torch.rand(100, 200) > 0.5).int()

###### **EXERCISE** 
>
> Given a binary tensor $X \in \{0, 1\}^{n \times m}$ return a tensor $y \in R^{n}$ that has in the $i$-th position the **number of ones** in the $i$-th row of $X$.

In [61]:
# Display a binary matrix with plotly

fig = px.imshow(x)
fig.show()

In [62]:
# ✏️ your code here

###### **Matrix-vector multiplication**

$$ c_i = \sum_k A_{ik}b_k := A_{ik}b_k $$

In [63]:
# Repeated indices in different input tensors indicate pointwise multiplication
a = torch.arange(6).reshape(2, 3)
b = torch.arange(3)
c = torch.einsum('ik, k -> i', [a, b])  # Multiply on k, then sum up on k
print_arr(a, b, c)

tensor([[0, 1, 2],
        [3, 4, 5]]) <shape: torch.Size([2, 3])> <dtype: torch.int64>

tensor([0, 1, 2]) <shape: torch.Size([3])> <dtype: torch.int64>

tensor([ 5, 14]) <shape: torch.Size([2])> <dtype: torch.int64>


###### **Matrix-matrix multiplication**

$$ C_{ij} = \sum_k A_{ik}B_{kj} := A_{ik}B_{kj} $$

![alt text](https://obilaniu6266h16.files.wordpress.com/2016/02/einsum-matrixmul.png?w=676)

In [64]:
a = torch.arange(6).reshape(2, 3)
b = torch.arange(15).reshape(3, 5)
c = torch.einsum('ik, kj -> ij', [a, b])
print_arr(a, b, c)

tensor([[0, 1, 2],
        [3, 4, 5]]) <shape: torch.Size([2, 3])> <dtype: torch.int64>

tensor([[ 0,  1,  2,  3,  4],
        [ 5,  6,  7,  8,  9],
        [10, 11, 12, 13, 14]]) <shape: torch.Size([3, 5])> <dtype: torch.int64>

tensor([[ 25,  28,  31,  34,  37],
        [ 70,  82,  94, 106, 118]]) <shape: torch.Size([2, 5])> <dtype: torch.int64>


###### **Dot product multiplication**

$$ c = \sum_i a_i b_i := a_i b_i $$

In [65]:
a = torch.arange(3)
b = torch.arange(3,6) 
c = torch.einsum('i,i->', (a, b))
print_arr(a, b, c)

tensor([0, 1, 2]) <shape: torch.Size([3])> <dtype: torch.int64>

tensor([3, 4, 5]) <shape: torch.Size([3])> <dtype: torch.int64>

tensor(14) <shape: torch.Size([])> <dtype: torch.int64>


###### **Point-wise multiplication**
Also known as hadamard product

$$ C_{ij} = A_{ij} B_{ij} $$

In [66]:
a = torch.arange(6).reshape(2, 3)
b = torch.arange(6,12).reshape(2, 3)
c = torch.einsum('ij, ij -> ij', (a, b))
print_arr(a, b, c)

tensor([[0, 1, 2],
        [3, 4, 5]]) <shape: torch.Size([2, 3])> <dtype: torch.int64>

tensor([[ 6,  7,  8],
        [ 9, 10, 11]]) <shape: torch.Size([2, 3])> <dtype: torch.int64>

tensor([[ 0,  7, 16],
        [27, 40, 55]]) <shape: torch.Size([2, 3])> <dtype: torch.int64>


###### **Outer product**

$$ C_{ij} = a_i b_j $$

In [67]:
a = torch.arange(3)
b = torch.arange(3,7)
c = torch.einsum('i, j -> ij', (a, b))
print_arr(a, b, c)

tensor([0, 1, 2]) <shape: torch.Size([3])> <dtype: torch.int64>

tensor([3, 4, 5, 6]) <shape: torch.Size([4])> <dtype: torch.int64>

tensor([[ 0,  0,  0,  0],
        [ 3,  4,  5,  6],
        [ 6,  8, 10, 12]]) <shape: torch.Size([3, 4])> <dtype: torch.int64>


In [68]:
# Using the standard PyTorch API
torch.ger(a, b)

tensor([[ 0,  0,  0,  0],
        [ 3,  4,  5,  6],
        [ 6,  8, 10, 12]])

###### **Batch matrix multiplication**

$$ c_{bij} = \sum_k a_{bik} b_{bkj} $$

In [69]:
a = torch.randn(2,2,5)
b = torch.randn(2,5,3)
c = torch.einsum('ijk,ikl->ijl', [a, b])
print_arr(a, b, c)

tensor([[[-1.3273, -1.5169, -0.1113,  0.5890, -0.2547],
         [ 0.3658,  1.1531, -2.0707,  0.3313,  1.0037]],

        [[-0.3241, -0.9605, -1.0997,  1.5115, -0.4926],
         [-0.7283,  0.0909,  0.1173, -0.5292, -0.9934]]]) <shape: torch.Size([2, 2, 5])> <dtype: torch.float32>

tensor([[[ 0.6091, -1.1030, -1.7355],
         [-1.1866,  0.8215, -0.8182],
         [ 0.5724, -0.7054,  0.1798],
         [ 1.0271, -0.6872, -1.6539],
         [-0.6724,  0.3927,  1.2095]],

        [[-0.5150,  1.6046,  2.0892],
         [ 1.1005,  0.2787, -0.5832],
         [-1.0956,  0.5278, -1.0307],
         [ 0.8401, -0.3870,  0.1877],
         [ 0.8952,  0.2783,  1.1534]]]) <shape: torch.Size([2, 5, 3])> <dtype: torch.float32>

tensor([[[ 1.7039, -0.2085,  2.2425],
         [-2.6654,  2.1711, -1.2845]],

        [[ 1.1435, -2.0902,  0.7321],
         [-0.9872, -1.1529, -2.9405]]]) <shape: torch.Size([2, 2, 3])> <dtype: torch.float32>


###### **EXERCISE**
> Implement:
> - Matrix transpose with einsum ($Y = M^T$)
> - Quadratic form with einsum  ($y = v^TMv$)

#### Singleton dimensions

 It is very common to **add or remove dimensions of size $1$** in a tensor.

 It is possible to perform these operations in different ways, feel free to use
 whatever is more comfortable to you. Again, **prefer readability to cryptic one-liners** for the sanity of a hypothetical unknown reader or your future self

e.g. If we want to transform a rank-1 tensor into a rank-2 column tensor and the back to a rank-1:

In [70]:
# Define a rank-1 tensor we will use
x = torch.arange(6)
print_arr(x)

tensor([0, 1, 2, 3, 4, 5]) <shape: torch.Size([6])> <dtype: torch.int64>


Transform **`x` into a column tensor** in four different ways.

Remember that the shape of a column tensor is in the form: `(rows, 1)`

In [71]:
# 1)
# Use the `reshape` or `view` functions

y1 = x.reshape(-1, 1)
y2 = x.view(-1, 1)

print_arr(y1, y2)

tensor([[0],
        [1],
        [2],
        [3],
        [4],
        [5]]) <shape: torch.Size([6, 1])> <dtype: torch.int64>

tensor([[0],
        [1],
        [2],
        [3],
        [4],
        [5]]) <shape: torch.Size([6, 1])> <dtype: torch.int64>


In [72]:
# 2)
# Use the specific `unsqueeze` function to unsqueeze a dimension

y3 = x.unsqueeze(dim=-1)
y4 = x.unsqueeze(dim=1)

print_arr(y3, y4)

tensor([[0],
        [1],
        [2],
        [3],
        [4],
        [5]]) <shape: torch.Size([6, 1])> <dtype: torch.int64>

tensor([[0],
        [1],
        [2],
        [3],
        [4],
        [5]]) <shape: torch.Size([6, 1])> <dtype: torch.int64>


In [73]:
# 3)
# Explicitly index a non-exixtent dimension with `None`

y5 = x[:, None]

print_arr(y5)

tensor([[0],
        [1],
        [2],
        [3],
        [4],
        [5]]) <shape: torch.Size([6, 1])> <dtype: torch.int64>


In [74]:
# 4)
# Same as before, but do not assume a bi-dimensional tensor.
# This approach is useful to write functions that work both for 
# batched or non-batched data

y6 = x[..., None]

print_arr(y5)

tensor([[0],
        [1],
        [2],
        [3],
        [4],
        [5]]) <shape: torch.Size([6, 1])> <dtype: torch.int64>


In [75]:
# To go back into a rank-1 tensor

x1 = y1.reshape(-1)
x2 = y2.view(-1)          # Explicity enforce to get a view of the tensors, without copying data
x3 = y3.squeeze(dim=-1)
x4 = y4.squeeze(dim=1)
x5 = y5[:, 0]             # Manually collapse the dimension with an integer indexing
x6 = y6[..., 0]

print_arr(x1, x2, x3, x4, x5, x6)

tensor([0, 1, 2, 3, 4, 5]) <shape: torch.Size([6])> <dtype: torch.int64>

tensor([0, 1, 2, 3, 4, 5]) <shape: torch.Size([6])> <dtype: torch.int64>

tensor([0, 1, 2, 3, 4, 5]) <shape: torch.Size([6])> <dtype: torch.int64>

tensor([0, 1, 2, 3, 4, 5]) <shape: torch.Size([6])> <dtype: torch.int64>

tensor([0, 1, 2, 3, 4, 5]) <shape: torch.Size([6])> <dtype: torch.int64>

tensor([0, 1, 2, 3, 4, 5]) <shape: torch.Size([6])> <dtype: torch.int64>


> **NOTE**
>
> indexing with `...` means  **keeps all the other dimension the same**.
> Keep in mind that `...` is just a Python singleton object (just as `None`). 
> Its type is Ellipsis:


In [76]:
...

Ellipsis

In [77]:
x = torch.rand(3,3,3)
x[:, :, 0]

tensor([[0.4184, 0.1653, 0.8977],
        [0.5616, 0.0259, 0.2847],
        [0.2687, 0.0114, 0.4334]])

In [78]:
x[..., 0]

tensor([[0.4184, 0.1653, 0.8977],
        [0.5616, 0.0259, 0.2847],
        [0.2687, 0.0114, 0.4334]])

### Tensor types
Pay attention to the tensor types!
Several methods are available to convert tensors to different types:

In [79]:
a = torch.rand(3, 3) + 0.5

In [80]:
a.int()

tensor([[1, 0, 1],
        [0, 1, 0],
        [0, 1, 1]], dtype=torch.int32)

In [81]:
a.long()

tensor([[1, 0, 1],
        [0, 1, 0],
        [0, 1, 1]])

In [82]:
a.float()

tensor([[1.3798, 0.5156, 1.3896],
        [0.6837, 1.2162, 0.9989],
        [0.5190, 1.4150, 1.2953]])

In [83]:
a.double()

tensor([[1.3798, 0.5156, 1.3896],
        [0.6837, 1.2162, 0.9989],
        [0.5190, 1.4150, 1.2953]], dtype=torch.float64)

In [84]:
a.bool()

tensor([[True, True, True],
        [True, True, True],
        [True, True, True]])

In [85]:
a.to(torch.double)

tensor([[1.3798, 0.5156, 1.3896],
        [0.6837, 1.2162, 0.9989],
        [0.5190, 1.4150, 1.2953]], dtype=torch.float64)

In [86]:
a.to(torch.uint8)

tensor([[1, 0, 1],
        [0, 1, 0],
        [0, 1, 1]], dtype=torch.uint8)

In [87]:
a.bool().int()

tensor([[1, 1, 1],
        [1, 1, 1],
        [1, 1, 1]], dtype=torch.int32)

---

---

---

Do not try to memorize all the PyTorch API: 

> Learn to understand what operation should already exist and search for it, when you need it. If it is something common, and it usually is, chances are it already exists.

Google, StackOverflow and the documentation are your friends!

### Einops

If you liked the `einsum` operation, have fun with the [einops](https://github.com/arogozhnikov/einops) package! 🚀

It is a third-party library, compatible with most frameworks, that brings superpowers to `einsum`. We will not use the `einops` library in the tutorials, however, feel free to read the [docs](https://github.com/arogozhnikov/einops) and use it.

![](http://arogozhnikov.github.io/images/einops/einops_video.gif)


### Final exercises


#### **EXERCISE 1**
>
> You are given $b$ images with dimensions $w \times h$. Each pixel in each image has an `(r, g, b)` $c$ channel. These images are organized in a tensor $X \in R^{w \times b \times c \times h}$.
>
> You want to apply a linear trasformation to the color channel of each single image. In particular, you want to **convert each image into a grey scale image**.
> **Afterthat, transpose the images** to swap the height and width.
>
> The linear traformation that converts from `(r, g, b)` to grey scale is simply a linear combination of `r`, `g` and `b`. It can be encoded in the following 1-rank tensor $y \in R^3$:

In [88]:
y = torch.tensor([0.2989, 0.5870, 0.1140], dtype=torch.float)


> You want to obtain a tensor $Z \in R^{b \times w \times h \times 3}$.
> 
> Write the PyTorch code that performs this operation.

In [89]:
# Create the input tensors for the exercise

from skimage import io
from skimage.transform import resize

size = 100

image1 = io.imread('https://upload.wikimedia.org/wikipedia/commons/thumb/6/6f/Earth_Eastern_Hemisphere.jpg/260px-Earth_Eastern_Hemisphere.jpg')
image1 = torch.from_numpy(resize(image1, (size, size), anti_aliasing=True)).float()  # Covert  to float type
image1 = image1[..., :3]  # remove alpha channel

image2 = io.imread('https://upload.wikimedia.org/wikipedia/commons/thumb/b/b4/The_Sun_by_the_Atmospheric_Imaging_Assembly_of_NASA%27s_Solar_Dynamics_Observatory_-_20100819.jpg/628px-The_Sun_by_the_Atmospheric_Imaging_Assembly_of_NASA%27s_Solar_Dynamics_Observatory_-_20100819.jpg')
image2 = torch.from_numpy(resize(image2, (size, size), anti_aliasing=True)).float()
image2 = image2[..., :3]  # remove alpha channel

image3 = io.imread('https://upload.wikimedia.org/wikipedia/commons/thumb/8/80/Wikipedia-logo-v2.svg/1920px-Wikipedia-logo-v2.svg.png')
image3 = torch.from_numpy(resize(image3, (size, size), anti_aliasing=True)).float()
image3 = image3[..., :3]  # remove alpha channel

source_images = torch.stack((image1, image2, image3), dim=0)
images = torch.einsum('bwhc -> wbch', source_images)

In [90]:
# Plot source images
plot_row_images(source_images)

In [91]:
# ✏️ your code here

In [92]:
# Plot the gray images
# plot_row_images(gray_images)

In [93]:
# Plot the gray transposed images
# plot_row_images(gray_images_tr)

#### **EXERCISE 2**
>
>  You are given $b$ images with dimensions $w \times h$. Each pixel in each image has an `(r, g, b)` $c$ channel. These images are organized in a tensor $X \in R^{w \times b \times c \times h}$, i.e. the same tensor as in the previous exercise.
>
> You want to swap the `red` color with the `blue` color, and decrese the intensity of the `green` by half
>
> Perform the transormation on all the images together simultaneusly.

In [94]:
images.shape

torch.Size([100, 3, 3, 100])

In [95]:
# ✏️ your code here

In [96]:
# plot_row_images(rb_images)

#### **EXERCISE 3**
>
> Given $k$ points organized in a tensor $X \in R^{k \times 2}$ apply a reflection along the $y$ axis as a linear transformation.
 

In [97]:
# Define some points in R^2
x = torch.arange(100, dtype=torch.float)
y = x ** 2

# Define some points in R^2
data = torch.stack((x, y), dim=0).t()

In [98]:
px.scatter(x = data[:, 0].numpy(), y = data[:, 1].numpy())  

In [99]:
# ✏️ your code here

In [100]:
# Plot the new points
# px.scatter(x = new_data[:, 0].numpy(), y = new_data[:, 1].numpy())  

#### **EXERCISE 4**
>
>  You are given $b$ images with dimensions $w \times h$. Each pixel in each image has an `(r, g, b)` $c$ channel. These images are organized in a tensor $X \in R^{w \times b \times c \times h}$, i.e. the same tensor as exercise 1 and 2.
>
> You want to **convert each image into a 3D point cloud**, where the `(x, y)`  coordinates are the indices of the pixels, and the `z` coordinate is the $L_2$ norm of the color of each pixel, multiplied by $10$
>
> *Hint*: you may need some other PyTorch function, search the docs!

In [101]:
# ✏️ your code here

In [102]:
from typing import Union

def plot_3d_point_cloud(cloud: Union[torch.Tensor, np.ndarray]) -> None:
  """ Plot a single 3D point cloud

  :param cloud: tensor with shape [number of points, coordinates]
  """
  import pandas as pd
  df = pd.DataFrame(np.asarray(cloud), columns=['x', 'y', 'z'])
  fig = px.scatter_3d(df, x=df.x, y=df.y, z=df.z, color=df.z, opacity=1, range_z=[0, 30])
  fig.update_layout({'scene_aspectmode': 'data', 'scene_camera':  dict(
          up=dict(x=0., y=0., z=0.),
          eye=dict(x=0., y=0., z=3.)
      )})
  fig.update_traces(marker=dict(size=3,),
                    selector=dict(mode='markers'))
  _ = fig.show()

In [103]:
# plot_3d_point_cloud(clouds[0, ...])

In [104]:
# plot_3d_point_cloud(clouds[1, ...])

In [105]:
# plot_3d_point_cloud(clouds[2, ...])